
# Linear Algebra Examples

**Learning goals**
1. See how linear algebra models real mechanical systems (spring–mass).
2. Use eigenvalues/eigenvectors to interpret **natural frequencies** and **mode shapes**.
3. Connect solving \(Kx = F\) to **least-squares regression** via the normal equations \(X^TX \beta = X^Ty\). We will learn more details about regressions in coming lectures.


In [1]:
import numpy as np
import matplotlib.pyplot as plt

# Ensure nice printing
np.set_printoptions(precision=4, suppress=True)

## Modeling a 2‑Mass, 3‑Spring System

**System:** 
Two masses (denoted as $m_1$ and $m_2$) are connected by three springs 
(with spring constants $k_1, k_2, k_3$) to walls and to each other.

<div style="text-align: center;">
    <img src="../../data/springs_and_masses.png" alt="Springs and Masses" width="800">
</div>

For this system, we will denote the displacements of the masses 
(with their respective equilibrium positions) as $x_1$ and $x_2$.

Then the equations of motion are given by:

$$
\begin{aligned}
m_1 \ddot x_1 + (k_1+k_2)\,x_1 - k_2\,x_2 &= 0,\\
m_2 \ddot x_2 + (k_2+k_3)\,x_2 - k_2\,x_1 &= 0.
\end{aligned}
$$

This can be written in matrix form as:

$$
\underbrace{\begin{bmatrix} m_1 & 0 \\ 0 & m_2 \end{bmatrix}}_{M}
\begin{bmatrix} \ddot x_1 \\ \ddot x_2 \end{bmatrix}
+
\underbrace{\begin{bmatrix} k_1+k_2 & -k_2 \\ -k_2 & k_2+k_3 \end{bmatrix}}_{K}
\begin{bmatrix} x_1 \\ x_2 \end{bmatrix}
= \mathbf{0}.
$$

with effective mass matrix $M$ and stiffness matrix $K$.

Let's further denote $\mathbf{x}(t)=[x_1(t),\,x_2(t)]^\top$, and
$\mathbf{z}=\begin{bmatrix}\mathbf{x}\\ \dot{\mathbf{x}}\end{bmatrix}\in\mathbb{R}^4$
as the state vector. Then the system can be written as:

$$
\dot{\mathbf{z}} = \underbrace{\begin{bmatrix} 0 & I \\ -M^{-1}K & 0 \end{bmatrix}}_{\mathbf{A}} \mathbf{z}.
$$

